<a href="https://colab.research.google.com/github/vorad1/CS703_dvor451/blob/working/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

from huggingface_hub import login
import os

# Get the secret
token = HF_TOKEN
login(token=token)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

# Loading FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Load CommonsenseQA dataset
dataset = load_dataset("commonsense_qa", cache_dir="/content/hf_cache")


In [3]:
# Selecting 2 in-context examples and 1 test question
prompt = """Question: Why would someone wear gloves?
A) To keep their hands clean
B) To make their hands soft
C) To show off
D) To cover scars
Answer: A

Question: Why do people use umbrellas?
A) To fly
B) To block the sun or rain
C) To walk faster
D) To play with
Answer: B

Question: Why do people wear sunglasses?
A) To look cool
B) To protect their eyes from bright light
C) To see better in the dark
D) To avoid being recognized
Answer:"""

# Generate prediction based on training
result = pipe(prompt, max_new_tokens=5)[0]['generated_text']
print("Model's answer:", result)


Model's answer: B
